<a href="https://colab.research.google.com/github/bettercodepaul/data-wrangling-praktikum/blob/master/dashBootstrapNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dash Einführung
Dash ist ein Framework zur Erstellung von Webapps für die Datenvisualisierung, entwickelt von Plotly. Dash zielt darauf ab, dass der User so weit wie möglich in Python arbeiten kann und eignet sich gut dafür, mit Pandas zusammen verwendet zu werden.

In diesem Notebook werden die Grundlagen von Dash anhand von Beispielen vorgestellt. Die offizielle Dokumentation befindet sich hier: 

https://dash.plotly.com/

*Anmerkung: Damit Dash in der Colab-Umgebung richtig funktioniert, sollten JavaScript und Cookies im Browser erlaubt sein.*

## Installation + Vorbereitung

In [ ]:
import urllib.request
import os.path

In [ ]:
REQUIREMENTS_URL = "https://github.com/bettercodepaul/data-wrangling-praktikum/raw/master/requirements.txt"
urllib.request.urlretrieve(REQUIREMENTS_URL, os.path.basename(REQUIREMENTS_URL))

In [ ]:
# nicht vergessen, dass die Laufzeitumgebung ggf. neu gestartet werden muss
!pip install -qr requirements.txt

In [ ]:
# Daten herunterladen
DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/spotify-charts-2017-2021-global-top200.csv.gz"
LOCAL_DATA_FILE_NAME = os.path.basename(DATA_URL)
#urllib.request.urlretrieve(DATA_URL, LOCAL_DATA_FILE_NAME)
GENRES_DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/track-genres.parquet"
LOCAL_GENRES_DATA_FILE_NAME = os.path.basename(GENRES_DATA_URL)
#urllib.request.urlretrieve(GENRES_DATA_URL, LOCAL_GENRES_DATA_FILE_NAME)
BIG_DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/releases/download/data-parquet/spotify-charts-2017-2021.parquet"
LOCAL_BIG_DATA_FILE_NAME = os.path.basename(BIG_DATA_URL)
#urllib.request.urlretrieve(BIG_DATA_URL, LOCAL_BIG_DATA_FILE_NAME)

In [ ]:
# Hilfsfunktionen herunterladen
EXERCISES_URL = "https://github.com/bettercodepaul/data-wrangling-praktikum/raw/master/utils_exercises.py"
urllib.request.urlretrieve(EXERCISES_URL, os.path.basename(EXERCISES_URL))

In [ ]:
from utils_exercises import *

In [ ]:
import polars as pl

In [ ]:
# bis zu 60 Zeichen pro Spalte ausgeben und Fließkommazahlen nicht abkürzen
pl.Config(fmt_str_lengths=60, fmt_float="full")

In [ ]:
df = df = (
    pl.read_csv(LOCAL_DATA_FILE_NAME, try_parse_dates=True)
)
df.head(2) # die ersten 2 Zeilen ausgeben

In [ ]:
region_df = (
    pl.read_parquet(LOCAL_BIG_DATA_FILE_NAME)
    .with_columns(pl.col("streams").cast(pl.Int64))
    .filter(pl.col("chart").eq("top200") & pl.col("region").ne("Global"))
)
# only keep top 25 regions
region_df = (
    region_df.join(
        region_df.group_by("region").agg(pl.sum("streams")).top_k(25, by="streams"),
        on="region", how="semi"
    )
)
region_df.head(2) # die ersten 2 Zeilen ausgeben

In [ ]:
from dash import dcc, html, Dash, _dash_renderer, Input, Output
import plotly.express as px
import dash_mantine_components as dmc
# React-Version auf 18.2.0 setzen (wird von Dash Mantine Components benötigt)
_dash_renderer._set_react_version("18.2.0")

## Statische Apps

Mit Dash können sehr aufwändige Oberflächen mit komplexen Diagrammen, Auswahl- und Filterelementen (z.B. Dropdown-Menüs, Checkboxen, Slider, etc.) und Layouts erstellt werden.

Normalerweise wird Dash als normales Pythonskript ausgeführt, der Zugriff auf die erzeugte Visualisierung erfolgt über den Browser. Innerhalb dieses Notebooks/der Colab-Umgebung greifen wir aber auf die Inline-Darstellung zurück.

In der Praxis könntet ihr eine Dash-App sehr einfach in einen Container packen und zum Beispiel in einem Kubernetes-Cluster oder Cloud-Service laufen lassen.

Für den Anfang erstmal eine minimale Dash-Anwendung:

In [ ]:
app = Dash(__name__)

# Das Layout wird ähnlich wie in HTML festgelegt
app.layout = html.Div(children=[
    html.H1(children="Hallo"),
    html.Div(children="Das ist Deine erste Dash-App.")
])

# Die Dash-App wird im Notebook selber angezeigt (jupyter_mode="inline")
app.run(jupyter_mode="inline", jupyter_height=150)

Für das Layout verwenden wir in diesem Notebook **Dash Mantine Components** (https://www.dash-mantine-components.com). Dies ändert nichts an den Konzepten von Dash, macht den Layout-Code bei etwas komplexeren Beispielen aber deutlich übersichtlicher und bringt mehr fertige und konsistent gestylte UI-Komponenten mit.

In [ ]:
app = Dash(external_stylesheets=dmc.styles.ALL)

# Wir müssen alle Komponenten in einem MantineProvider einschließen
app.layout = dmc.MantineProvider([
     dmc.Title("Hallo", order=1),
     dmc.Text("Das ist Deine erste Dash-App mit Mantine.")
])

# Die Dash-App wird im Notebook selber angezeigt (jupyter_mode="inline")
app.run(jupyter_mode="inline", jupyter_height=150)

Um Diagramme für unsere Daten erstellen zu können verwenden wir das `plotly.express` Modul, oben importiert als `px`. Es ermöglicht die Erstellung interaktiver Diagramme im Browser (das oft benutzte Matplotlib dagegen erzeugt nur statische Diagramme): https://plotly.com/python/plotly-express/

In [ ]:
# Balkendiagramm erzeugen
k = 10
top_k_regions = (
    region_df
    .lazy()
    .group_by("region")
    .agg(pl.col("streams").sum())
    .select(
        "region",
        pl.when(pl.col("streams").rank(descending=True).lt(k))
        .then(pl.col("region"))
        .otherwise(pl.lit("Others"))
        .alias("top_k_region")
    )
)
streams_per_month = (
    region_df
    .lazy()
    .join(top_k_regions, on="region")
    .group_by(
        "top_k_region",
        pl.col("date").dt.month_start()
    )
    .agg(pl.col("streams").sum())
    .collect()
)
streams_per_month.sample(3)

In [ ]:
px.bar(streams_per_month, x="date", y="streams", color="top_k_region", height=300)

Mit einem `dcc.Graph` können wir das Diagramm auch in einer Dash-App anzeigen.

In [ ]:
app = Dash(external_stylesheets=dmc.styles.ALL)

app.layout = dmc.MantineProvider([
     dmc.Title("Dash-App mit Diagramm", order=5),
     # Diagramm wird mit dem Attribut figure übergeben
     dcc.Graph(id="streams-chart", figure=px.bar(streams_per_month, x="date", y="streams", color="top_k_region", height=300))
])

app.run(jupyter_mode="inline", jupyter_height=350)

### Übung

Jetzt zu den Aufgaben für Dich:



In [ ]:
q1.question()

In [ ]:
q1_df = ...

In [ ]:
q1.check(q1_df)

In [ ]:
q2.question()


In [ ]:
q2_fig = ...

In [ ]:
q2.check(q2_fig)

## Interaktive Apps mit Callbacks
Ein höherer Grad an Interaktion wird in Dash über Callbacks erreicht. Mit Callbacks kann auf Userinputs reagiert werden, um zum Beispiel bei der Auswahl in einer Dropdown-Komponente eine andere UI-Komponenten wie ein Diagramme anzupassen. Auch hierzu wieder ein Minimalbeispiel:

In [ ]:
app = Dash(external_stylesheets=dmc.styles.ALL)

# Für Callbacks sind die ids der Komponenten wichtig
app.layout = dmc.MantineProvider([
     dmc.TextInput(id="my-input", value=None, label="Deine Eingabe", placeholder="Hier beliebigen Text eingeben..."),
     dmc.TextInput(id="my-output", value=None, label="Meine Ausgabe", disabled=True)
])


# Der Callback wird vom Client aufgerufen, sobald sich eine Property des Inputs ändert
# Die Rückgabe des Callbacks wird vom Client in die Property des Outputs geschrieben
@app.callback(
    Output(component_id="my-output", component_property="value"),
    Input(component_id="my-input", component_property="value")
)
def update_output_div(input_value):
    output_value = input_value
    return output_value

app.run(jupyter_mode="inline", jupyter_height=150)

Callbacks werden in Dash mit `@app.callback` und den Output- und Inputparametern annotiert. Die Annotation verknüpft die Callbackfunktion über ID- und Property-Werte mit den entsprechenden Elementen, die im Layout spezifiziert sind. Alle Callbacks werden beim Start der Anwendung einmal ausgeführt, um Elemente initial einzurichten.

<pre>
# Für Callbacks sind die ids der Komponenten wichtig
app.layout = dmc.MantineProvider([
     dmc.TextInput(id=<span style="color:#4169E1">"my-input"</span>, <span style="color:#4169E1">value</span>=<span style="color:white; background-color:#4169E1">None</span>, label="Deine Eingabe", placeholder="Hier beliebigen Text eingeben..."),
     dmc.TextInput(id=<span style="color:#C04000">"my-output"</span>, <span style="color:#C04000">value</span>=<span style="color:white; background-color:#C04000">None</span>, label="Meine Ausgabe", disabled=True)
])


# Der Callback wird vom Client aufgerufen, sobald sich eine Property des Inputs ändert
# Die Rückgabe des Callbacks wird vom Client in die Property des Outputs geschrieben
@app.callback(
    Output(component_id=<span style="color:#C04000">"my-output"</span>, component_property=<span style="color:#C04000">"value"</span>),
    Input(component_id=<span style="color:#4169E1">"my-input"</span>, component_property=<span style="color:#4169E1">"value"</span>)
)
def update_output_div(<span style="color:white; background-color:#4169E1">input_value</span>):
    output_value = input_value
    return <span style="color:white; background-color:#C04000">output_value</span>
</pre>

Über Properties werden die Attribute der Layout-Elemente spezifiziert, die für die Callback-Funktion relevant sind. Im Output können auch mehrere Attribute gleichzeitig verändert werden, ein Beispiel hierzu kommt am Ende des Notebooks.

Mithilfe von Callbacks bauen wir nun eine einfache Filtermöglichkeit ein:

In [ ]:
all_artists = region_df.get_column("artist").unique().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

def get_streams_per_month_figure(selected_artist):
    if selected_artist is not None:
        filter_expr = pl.col("artist").str.contains(selected_artist)
    else:
        filter_expr = pl.lit(True)
    k = 10
    top_k_regions = (
        region_df
        .lazy()
        .filter(filter_expr)
        .group_by("region")
        .agg(pl.col("streams").sum())
        .select(
            "region",
            pl.when(pl.col("streams").rank(descending=True).lt(k))
            .then(pl.col("region"))
            .otherwise(pl.lit("Others"))
            .alias("top_k_region")
        )
    )
    streams_per_month = (
        region_df
        .lazy()
        .filter(filter_expr)
        .join(top_k_regions, on="region")
        .group_by(
            "top_k_region",
            pl.col("date").dt.month_start()
        )
        .agg(pl.col("streams").sum())
        .collect()
    )
    return px.bar(streams_per_month, x="date", y="streams", color="top_k_region", height=300, title=f"Monthly Streams for {selected_artist}")

app.layout = dmc.MantineProvider([
    dmc.Autocomplete(id="artist-select", label="Artist", placeholder="Select one", data=all_artists, w="400", mb=10),
    dcc.Graph(id="streams-chart", figure=get_streams_per_month_figure(None))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Input(component_id="artist-select", component_property="value")
)
def update_streams_per_month(selected_artist):
    return get_streams_per_month_figure(selected_artist)

app.run(jupyter_mode="inline", jupyter_height=400)

### Mehrere Inputs

Nur nach dem Künstler zu filtern reicht vielleicht nicht, deshalb wollen wir zusätzlich nach Region filtern können und auch die Anzahl der angezeigten Top-Regionen bestimmen können.

In [ ]:
# Zuerst eine Hilfs-Funktion die die Streams pro Monat gefiltert nach Künstler und Region zurückgibt
def get_streams_per_month_figure(selected_artist, regions, top_k=10):
    if selected_artist is not None:
        filter_expr = pl.col("artist").str.contains(selected_artist)
    else:
        filter_expr = pl.lit(True)
    if regions is not None and len(regions) > 0:
        filter_expr &= pl.col("region").is_in(regions)
    top_k_regions = (
        region_df
        .lazy()
        .filter(filter_expr)
        .group_by("region")
        .agg(pl.col("streams").sum())
        .select(
            "region",
            pl.when(pl.col("streams").rank(descending=True).lt(top_k))
            .then(pl.col("region"))
            .otherwise(pl.lit("Others"))
            .alias("top_k_region")
        )
    )
    streams_per_month = (
        region_df
        .lazy()
        .filter(filter_expr)
        .join(top_k_regions, on="region")
        .group_by(
            "top_k_region",
            pl.col("date").dt.month_start()
        )
        .agg(pl.col("streams").sum())
        .collect()
    )
    return px.bar(streams_per_month, x="date", y="streams", color="top_k_region", height=300, title=f"Monthly Streams for {selected_artist}")

Implementierung in der Dash-App mit zusätzlichen Input-Komponenten.

In [ ]:
all_artists = region_df.get_column("artist").unique().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

app.layout = dmc.MantineProvider([
    dmc.Group([
        dmc.Autocomplete(id="artist-select", label="Artist", placeholder="Select one", data=all_artists),
        dmc.NumberInput(id="top-k-input", label="Show top k regions", value=10),
        dmc.MultiSelect(id="region-select", label="Regions", data=region_df.get_column("region").unique().to_list(), searchable=True, debounce=True),
    ], grow=True, w="100%"),
    dcc.Graph(id="streams-chart", figure=get_streams_per_month_figure(None, None))
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="region-select", component_property="value")
)
def update_streams_per_month(selected_artist, regions):
    return get_streams_per_month_figure(selected_artist, regions)

app.run(jupyter_mode="inline", jupyter_height=400)

### Übung

In [ ]:
q3.question()

In [ ]:
# Platz für deine Lösung

In [ ]:
#q3.hint()

## IFrame

Damit wir die Lieder auch anhören können, bauen wir einen IFrame mit dem Spotify Player ein.

In [ ]:
def get_top_song_player_url(selected_artist, regions):
    if selected_artist is not None:
        filter_expr = pl.col("artist").str.contains(selected_artist)
    else:
        filter_expr = pl.lit(True)
    if regions is not None and len(regions) > 0:
        filter_expr &= pl.col("region").is_in(regions)
    top_song = (
        region_df
        .lazy()
        .filter(filter_expr)
        .group_by(
            "url",
        )
        .agg(pl.col("streams").sum())
        .top_k(1, by="streams")
        .collect()
    )
    return play_song(top_song)

In [ ]:
all_artists = region_df.get_column("artist").unique().to_list()

app = Dash(external_stylesheets=dmc.styles.ALL)

app.layout = dmc.MantineProvider([
    dmc.Group([
        dmc.Autocomplete(id="artist-select", label="Artist", placeholder="Select one", data=all_artists),
        dmc.NumberInput(id="top-k-input", label="Show top k regions", value=10),
        dmc.MultiSelect(id="region-select", label="Regions", data=region_df.get_column("region").unique().to_list(), searchable=True, debounce=True),
    ], grow=True, w="100%"),
    dcc.Graph(id="streams-chart", figure=get_streams_per_month_figure(None, None)),
    dmc.Title("Most streamed title", order=4, mb=10),
    html.Iframe(id="player-iframe", src=None, width="100%", height="80", style={"border": "none"})
])

@app.callback(
    Output(component_id="streams-chart", component_property="figure"),
    Output(component_id="player-iframe", component_property="src"),
    Input(component_id="artist-select", component_property="value"),
    Input(component_id="region-select", component_property="value")
)
def update_streams_per_month(selected_artist, regions):
    spotify_url = get_top_song_player_url(selected_artist, regions).src
    return (get_streams_per_month_figure(selected_artist, regions), spotify_url)

app.run(jupyter_mode="inline", jupyter_height=600)

## Weiterführende Ressourcen

*  Dash Gallery (Umfangreiche Beispielsammlung für Dash-Apps, inkl. Source-Code): https://dash-gallery.plotly.host/Portal/

* lokale Python-Umgebung: Anaconda (https://www.anaconda.com/products/individual#Downloads)
* lokale Python-IDE: VSCode (https://code.visualstudio.com/)
* Zugriff auf Datenbanken: SQLAlchemy (conda install sqlalchemy), (https://www.sqlalchemy.org/)
